In [ ]:
#| default_exp target_transforms

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# Target transforms
Transformations that can be applied to the target before fitting and restored after predicting.

In [ ]:
#| export
import math
from typing import Callable, Tuple

try:
    from numba import njit
except ImportError:
    raise ImportError(
        "Please install numba. "
        "You can find detailed instructions at https://numba.pydata.org/numba-doc/latest/user/installing.html"
    )
import numpy as np

from utilsforecast.grouped_array import GroupedArray

In [ ]:
#| exporti
@njit
def _fit(
    data: np.ndarray,
    indptr: np.ndarray,
    stats_fn: Callable,
) -> np.ndarray:
    n_groups = len(indptr) - 1
    stats = np.empty((n_groups, 2), dtype=data.dtype)
    for i in range(n_groups):
        sl = slice(indptr[i], indptr[i + 1])
        stats[i] = stats_fn(data[sl])   
    return stats


@njit
def _transform(
    data: np.ndarray,
    indptr: np.ndarray,
    stats: np.ndarray,
    tfm_fn: Callable,
) -> np.ndarray:
    n_groups = len(indptr) - 1
    out = np.empty_like(data)
    for i in range(n_groups):
        sl = slice(indptr[i], indptr[i + 1])
        offset, scale = stats[i]
        out[sl] = tfm_fn(data[sl], offset, scale)
    return out


@njit
def _common_scaler_transform(data: np.ndarray, offset: float, scale: float) -> np.ndarray:
    if abs(scale) < np.finfo(data.dtype).eps:
        return data
    return (data - offset) / scale


@njit
def _common_scaler_inverse_transform(data: np.ndarray, offset: float, scale: float) -> np.ndarray:
    if abs(scale) < np.finfo(data.dtype).eps:
        return data
    return data * scale + offset

In [ ]:
#| export
class BaseTargetTransform:
    """Base class used for target transformations."""
    stats_fn: Callable
    
    def fit(self, ga: GroupedArray) -> 'BaseTargetTransform':
        self.stats_ = _fit(ga.data, ga.indptr, self.stats_fn)
        return self

    def transform(self, ga: GroupedArray) -> np.ndarray:
        return _transform(ga.data, ga.indptr, self.stats_, _common_scaler_transform)    
    
    def fit_transform(self, ga: GroupedArray) -> np.ndarray:
        return self.fit(ga).transform(ga)
    
    def inverse_transform(self, ga: GroupedArray) -> np.ndarray:
        return _transform(ga.data, ga.indptr, self.stats_, _common_scaler_inverse_transform)

In [ ]:
#| exporti
@njit
def _standard_scaler_stats(data: np.ndarray) -> Tuple[float, float]:
    return np.nanmean(data), np.nanstd(data)

In [ ]:
#| export
class LocalStandardScaler(BaseTargetTransform):
    """Standardizes each serie by subtracting its mean and dividing by its standard deviation."""
    stats_fn = _standard_scaler_stats    

In [ ]:
from utilsforecast.data import generate_series

In [ ]:
series = generate_series(10, min_length=50, max_length=100)
data = series['y'].values
sizes = series.groupby('unique_id', observed=True).size().values
indptr = np.append(0, sizes.cumsum())
ga = GroupedArray(data, indptr)
# introduce some constant series
for i, c in zip([0, 3], [10, 0]):
    ga.data[ga.indptr[i] : ga.indptr[i + 1]] = c * np.ones(ga[i].size)

In [ ]:
def test_transform(tfm, ga):
    transformed = tfm.fit_transform(ga)
    transformed2 = tfm.transform(ga)
    np.testing.assert_allclose(transformed, transformed2)
    transformed_ga = GroupedArray(transformed, ga.indptr)
    np.testing.assert_allclose(
        tfm.inverse_transform(transformed_ga),
        ga.data,
    )

In [ ]:
test_transform(LocalStandardScaler(), ga)

In [ ]:
#| exporti
@njit
def _minmax_scaler_stats(data: np.ndarray) -> Tuple[float, float]:
    min_ = np.nanmin(data)
    max_ = np.nanmax(data)
    return min_, max_ - min_

In [ ]:
#| export
class LocalMinMaxScaler(BaseTargetTransform):
    """Scales each serie to be in the [0, 1] interval."""
    stats_fn = _minmax_scaler_stats

In [ ]:
test_transform(LocalMinMaxScaler(), ga)

In [ ]:
#| exporti
@njit
def _robust_scaler_iqr_stats(data: np.ndarray) -> Tuple[float, float]:
    q25, median, q75 = np.nanquantile(data, (0.25, 0.5, 0.75))
    return median, q75 - q25


@njit
def _robust_scaler_mad_stats(data: np.ndarray) -> Tuple[float, float]:
    median = np.nanmedian(data)
    mad = np.nanmedian(np.abs(data - median))
    return median, mad

In [ ]:
#| export
class LocalRobustScaler(BaseTargetTransform):
    """Scaler robust to outliers.

    Parameters
    ----------
    scale : str (default='iqr')
        Statistic to use for scaling. Can be either 'iqr' (Inter Quartile Range) or 'mad' (Median Asbolute Deviation)
    """
    def __init__(self, scale: str = 'iqr'):
        supported_scales = ('iqr', 'mad')
        if scale not in supported_scales:
            raise ValueError(f'scale must be one of {supported_scales}')
        self.scale = scale

    def fit(self, ga: GroupedArray) -> 'LocalRobustScaler':
        if self.scale == 'iqr':
            stats_fn = _robust_scaler_iqr_stats
        else:
            stats_fn = _robust_scaler_mad_stats
        self.stats_ = _fit(ga.data, ga.indptr, stats_fn)
        return self

In [ ]:
for scale in ('iqr', 'mad'):
    test_transform(LocalRobustScaler(scale=scale), ga)

In [ ]:
#| export
class LocalBoxCox(BaseTargetTransform):
    """Finds optimum lambda for each serie and applies Box-Cox transformation."""
    def fit_transform(self, ga: GroupedArray) -> np.ndarray:
        from scipy.stats import boxcox

        out = np.full(ga.data.shape, np.nan)
        self.lmbdas_ = np.empty(ga.n_groups)
        for i in range(ga.n_groups):
            sl = slice(ga.indptr[i], ga.indptr[i + 1])
            mask = ~np.isnan(ga.data[sl])
            try:
                transformed, self.lmbdas_[i] = boxcox(ga.data[sl][mask] + 1.0, lmbda=None)
            except ValueError:
                # this can happen for constant series, fallback to log
                self.lmbdas_[i] = 0.0
                transformed = np.log1p(ga.data[sl][mask])
            if not math.isclose(self.lmbdas_[i], 0.0) and np.isclose(transformed * self.lmbdas_[i], -1).any():
                # in this case we can't reliably invert the transformation
                # fallback to log
                self.lmbdas_[i] = 0.0
                transformed = np.log1p(ga.data[sl][mask])
            out[sl][mask] = transformed
        return out

    def transform(self, ga: GroupedArray) -> np.ndarray:
        from scipy.special import boxcox1p
        
        sizes = np.diff(ga.indptr)
        lmbdas = np.repeat(self.lmbdas_, sizes, axis=0)
        return boxcox1p(ga.data, lmbdas)

    def inverse_transform(self, ga: GroupedArray) -> np.ndarray:
        from scipy.special import inv_boxcox1p

        sizes = np.diff(ga.indptr)
        lmbdas = np.repeat(self.lmbdas_, sizes, axis=0)
        return inv_boxcox1p(ga.data, lmbdas)

In [ ]:
test_transform(LocalBoxCox(), ga)

In [ ]:
#| export
class GlobalFuncTransformer(BaseTargetTransform):
    """Uses `func` and `inverse_func` for applying the same transformation to all series.

    Parameters
    ----------
    func : Callable
        Function that transforms the data.
    inverse_func : Callable
        Function that inverse transforms the data.
    """
    def __init__(self, func: Callable, inverse_func: Callable):
        self.func = func
        self.inverse_func = inverse_func

    def transform(self, ga: GroupedArray) -> np.ndarray:
        return self.func(ga.data)
        
    def fit_transform(self, ga: GroupedArray) -> np.ndarray:
        return self.transform(ga)

    def inverse_transform(self, ga: GroupedArray) -> np.ndarray:
        return self.inverse_func(ga.data)

In [ ]:
test_transform(GlobalFuncTransformer(np.log1p, np.expm1), ga)